In [1]:
# pip install pandas_datareader

In [2]:
#pip install yfinance

In [3]:
#pip install keras

In [ ]:
# Libs Necessárias
import numpy as np
import pandas as pd
import pandas_datareader.data as web

# Libs para gráficos
import matplotlib.pyplot as plt

# Libs para utilizar o Plotly
import plotly.express as px
import plotly.graph_objects as Dash

# pip install yfinance
import yfinance as yf
yf.pdr_override()

# Libs para uso de Machine Learning do Keras
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Ignorando avisos
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Pegar os dados do Bitcoin com o Yfinance
Base_Dados = web.get_data_yahoo( 'BTC-USD', start=‘2021-01-01' )

# Verificando
Base_Dados.head()

In [ ]:
# Plot para verificar preço de fechamento
# Chamando o gráfico
fig = px.line( Base_Dados, y='Close' )
fig.show()

In [ ]:
# --- Gráficos com as médias
# Gerar a média movel do fechamento das ações
Media_Movel = Base_Dados['Close'].rolling(5).mean()
Media_Movel_Tendencia = Base_Dados['Close'].rolling(30).mean()

# Definindo uma figura
Figura = Dash.Figure()

# Incluindo o Eixo no Gráfico - Abertura
Figura.add_trace(Dash.Scatter(x = Base_Dados.index, y = Base_Dados.Close,mode='lines', name='Fechamento', marker_color = '#FF7F0E'))

# Incluindo o Eixo no Gráfico - Maior
Figura.add_trace(Dash.Scatter(x = Base_Dados.index, y = Media_Movel,mode='lines', name='Média Móvel', opacity=0.5,marker_color = '#2CA02C',))

# Incluindo o Eixo no Gráfico - Menor
Figura.add_trace(Dash.Scatter(x = Base_Dados.index, y = Media_Movel_Tendencia,mode='lines', name='Tendência', opacity=0.5,marker_color = '#D62728'))

# Modificando o Layout do Gráfico
Figura.update_layout(title='Histórico de Preço', # Titulo
                     titlefont_size = 28, # Tamanho da Fonte
                    # Parametros para mexer no eixo X
                    xaxis = dict(title='Período Histórico', # Titulo do Eixo x
                                 titlefont_size=16, # Tamanho fonte do Titulo
                                 tickfont_size=14), # Tamanho da fonte do eixo
                    # Tamanho do Grafico
                    height = 500,
                    # Parametros para mexer no eixo y
                    yaxis=dict(
                                title='Preço Bitcoint', # Titulo do Eixo y
                                titlefont_size=16, # Tamanho fonte do Titulo
                                tickfont_size=14), # Tamanho da fonte do eixo
                    # Parametros para mexer na legenda
                    legend=dict(
                                y=1, x=1, # Posição da Legenda
                                bgcolor='rgba(255, 255, 255, 0)', # Cor de fundo
                                bordercolor='rgba(255, 255, 255, 0)')) # Cor da Bornda
# Mostrando o Gráfico
Figura.show()

In [ ]:
# Estatísticas
Base_Dados.describe()

In [ ]:
# Função para escolanemnto
from sklearn.preprocessing import MinMaxScaler

# Chamando a Função
Funcao_MinMAx = MinMaxScaler( feature_range=(0,1) )

# Criando uma copia
Dados_Treino = Base_Dados.filter(['Close'])

# Aplicando a função
Dados_Treino_Escalados = Funcao_MinMAx.fit_transform( Dados_Treino )

# VErificando
Dados_Treino_Escalados[0:5]

In [ ]:
# Listas para receber os dados
x_treinamento = []
y_treinamento = []

# Loop para separar os dados de treino e teste
# Nesse Loop vamos seprar os dados em blocos de 60 valores
for Loop in range( 60, len(Dados_Treino_Escalados) ):
    # Separando os dados de treinamento x
    Filtrando_Amostra_Treinamento_x = Dados_Treino_Escalados[Loop-60 : Loop, 0 ]
    x_treinamento.append( Filtrando_Amostra_Treinamento_x )
    # Separando os dados de treinamento y
    Filtrando_Amostra_Treinamento_y = Dados_Treino_Escalados[Loop, 0]
    y_treinamento.append( Filtrando_Amostra_Treinamento_y )
# Transformando as listas em Array
x_treinamento, y_treinamento = np.array(x_treinamento), np.array(y_treinamento)
# Convertendo o array para Matriz
x_treinamento = np.reshape(x_treinamento, (x_treinamento.shape[0], x_treinamento.shape[1], 1))
# Verificando a demisão da nossa matriz
x_treinamento.shape

In [ ]:
# Definindo a função do Keras
# Essa função é uma pilha linear de camadas do Keras
Modelo = Sequential()

# Adicinando as camadas e parametros para nossa rede neural
# Treinamento da Rede Neural Recorrente
# LSTM - Long Short-Term Memory
Modelo.add(LSTM(50, return_sequences = True,input_shape = (x_treinamento.shape[1], 1)))
Modelo.add(LSTM(50, return_sequences = False))

# Adicionando as camadas na rede neural
Modelo.add(Dense(25))
Modelo.add(Dense(1))
Modelo.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Treinando o modelo
Modelo.fit(x_treinamento, y_treinamento, batch_size = 1, epochs = 10)

In [ ]:
import math

# Arredondando o numero para cima usando o 'math.ceil'
Dados_Fechamento_Valores_Tamanho = math.ceil( len(Dados_Treino) * .8)
# Definindo amostra para ser testada
Dados_Teste = Dados_Treino_Escalados[Dados_Fechamento_Valores_Tamanho - 60: , :]

# Lista para recever os dados de teste
x_teste = []
# Lista com os dados de teste
y_Teste = Dados_Treino_Escalados[Dados_Fechamento_Valores_Tamanho:, :]

# Loop para fixar amostra para teste
for Loop in range (60, len(Dados_Teste)):
    x_teste.append(Dados_Teste[Loop - 60:Loop, 0])
    # Transformando os dados em um array
    x_teste = np.array(x_teste)
    # Convertendo o array para Matriz
    x_teste = np.reshape(x_teste, (x_teste.shape[0], x_teste.shape[1], 1))
    # Aplicando as Previsões
    Previsoes = Modelo.predict(x_teste)
    # Calculando o erro quadrático médio
    rsme = np.sqrt(np.mean(Previsoes - y_Teste) ** 2)
    print('Erro Quadrático Médio:', rsme )
# Invertendo para escalas reais
Previsoes = Funcao_MinMAx.inverse_transform(Previsoes)

In [ ]:
# Criando a base para verificar o real x modelo
Validação = Dados_Treino[Dados_Fechamento_Valores_Tamanho:]

# Atribuindo as previsões no DataSet
Validação['Previsões'] = Previsoes

# Definindo uma figura
Figura = Dash.Figure()

# Incluindo o Eixo no Gráfico - Fechamento
Figura.add_trace(Dash.Scatter(x = Validação.index, y = Validação.Close,mode='lines', name='Fechamento', marker_color = '#FF7F0E', ))

# Incluindo o Eixo no Gráfico - Previsão
Figura.add_trace(Dash.Scatter(x = Validação.index, y = Validação.Previsões,mode='lines', name='Previsão', marker_color = '#2CA02C', ))

# Modificando o Layout do Gráfico
Figura.update_layout( title='Realizado vs Modelo', # Titulo
                     titlefont_size = 28, # Tamanho da Fonte
                    # Parametros para mexer no eixo X
                    xaxis = dict(
                                title='Período Histórico', # Titulo do Eixo x
                                titlefont_size=16, # Tamanho fonte do Titulo
                                tickfont_size=14), # Tamanho da fonte do eixo
                    # Tamanho do Grafico
                    height = 500,
                    # Parametros para mexer no eixo y
                    yaxis=dict(
                                title='Preço do Bitcoin', # Titulo do Eixo y
                                titlefont_size=16, # Tamanho fonte do Titulo
                                tickfont_size=14), # Tamanho da fonte do eixo
                    # Parametros para mexer na legenda
                    legend=dict(
                                y=1, x=1, # Posição da Legenda
                                bgcolor='rgba(255, 255, 255, 0)', # Cor de fundo
                                bordercolor='rgba(255, 255, 255, 0)')) # Cor da Bornda
# Mostrando o Gráfico
Figura.show()